# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openai import OpenAI
from scipy.spatial.distance import cosine
import tiktoken

In [2]:
OPENAI_API_KEY = 'YOUR_API_KEY'
EMBEDDINGS_MODEL = 'text-embedding-3-small'
GPT_MODEL = 'gpt-4o-mini-2024-07-18'
BASE_URL = 'https://openai.vocareum.com/v1'

In [3]:
#Website of the page to scrap
url = 'https://theanalyst.com/eu/2024/05/premier-league-best-facts-2023-24'
file = 'premier_league_2023-24.html'

In [4]:
#Function to fetch html page from the given URL
def fetch_html_page(url):
    
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        raise Exception('Connection error')

with open(file, mode='wb') as html_file:
    html_page = fetch_html_page(url)
    html_file.write(html_page) 

In [5]:
#Function to extract data from the given html page and return it as a pandas dataframe
def extract_data(html_file):
    with open(html_file) as fp:
        soup = BeautifulSoup(fp, 'html.parser')

    nodes = soup.find('h2', {'class': 'wp-block-heading has-text-align-center'})
    month_tags = []
    text = []
    current_month = None
    for month in soup.find_all('h2', {'class': 'wp-block-heading has-text-align-center'}):
        month_tags.append(month.find_next('strong'))

    for node in nodes.find_all_next():
        if node in month_tags:
            current_month = node.text

        elif node.name == 'ul':
            tags = node.find_all('li')
            for tag in tags:
                event = tag.text.strip()
                text.append(f'{current_month}: {event}') 

    return pd.DataFrame(text, columns=['text'])

#Extracting the data and adding it to a dataframe
df = extract_data(file)
pd.set_option('display.max_colwidth', None)
df.head()

,text
0,"August: With his brace against Burnley, Erling Haaland became only the second player to score 2+ goals in a team’s opening game of a Premier League season in consecutive campaigns (also two vs West Ham in 2022-23), after Didier Drogba in 2009-10 (two v Hull) and 2010-11 (three v West Brom) for Chelsea."
1,"August: Vincent Kompany became the first Belgian to manage in the Premier League, while he became just the third manager to face a team he previously played for in the competition in his maiden fixture (also Roberto Di Matteo for West Brom v Chelsea in August 2010 and Scott Parker for Fulham v Chelsea in March 2019), with all three losing those games."
2,"August: James Milner appeared in his 22nd Premier League season, equalling the competition’s record held by Ryan Giggs."
3,"August: Luton became the sixth side whose first ever Premier League goal was a penalty, and first since Swansea in September 2011."
4,August: Aston Villa’s 5-1 loss at Newcastle on MD1 was the first time they had conceded 5+ in their first league match since 1951-52 (2-5 v Bolton Wanderers).


In [6]:
#To check total number of token in the dataset based on the gpt model
encoding = tiktoken.encoding_for_model(GPT_MODEL) 

total_text = " ".join(df['text'].tolist())
tokens = encoding.encode(total_text)
num_tokens = len(tokens)

print(f"Total number of tokens: {num_tokens}")

Total number of tokens: 6724


In [8]:
#Initializing OpenAI Client
client = OpenAI(base_url=BASE_URL, api_key=OPENAI_API_KEY)

In [9]:
#Function to get embeddings for the given text based on the given embeddings model
def get_embeddings(text, model):
    response = client.embeddings.create(input=text, model=EMBEDDINGS_MODEL)
    return [data.embedding for data in response.data]

#Function to create embeddings for the given dataframe based on the given embeddings model
def create_embeddings(df, model):
    embeddings = []
    batch_size = 50
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i : i + batch_size]['text'].tolist()
        embeddings.extend(get_embeddings(batch, model))
    return embeddings

In [10]:
#Creating embeddings for the dataset and adding it to a embeddings column
df['embeddings'] = create_embeddings(df, EMBEDDINGS_MODEL)

In [11]:
#Saving the dataframe with embeddings to a csv file
df.to_csv('text_embeddings.csv')

In [12]:
df.head()

,text,embeddings
0,"August: With his brace against Burnley, Erling Haaland became only the second player to score 2+ goals in a team’s opening game of a Premier League season in consecutive campaigns (also two vs West Ham in 2022-23), after Didier Drogba in 2009-10 (two v Hull) and 2010-11 (three v West Brom) for Chelsea.","[-0.0053741345182061195, -0.00987408310174942, 0.09268473088741302, -0.006345744244754314, 0.043646082282066345, 0.014713700860738754, 0.017589032649993896, 0.01740998402237892, 0.011259088292717934, 0.011111634783446789, -0.026688989251852036, -0.007493770681321621, -0.022518176585435867, 0.004739559721201658, 0.007314720656722784, 0.035788942128419876, -0.0344829298555851, -0.010837793350219727, -0.05847563594579697, -0.011680382303893566, -0.0275315772742033, 0.0031412749085575342, 0.006114032119512558, 0.03361927717924118, 0.018336830660700798, 0.0176100991666317, -0.0026949665043503046, -0.0139448381960392, -0.02833203598856926, 0.019621778279542923, 0.032397523522377014, -0.03214474767446518, 0.06690151989459991, 0.004394625313580036, 0.05666407197713852, 0.018673866987228394, -0.0004114201292395592, -0.004405157640576363, 0.035051677376031876, -0.024203352630138397, 0.004555243533104658, -0.03536764904856682, -0.0023079023230820894, -0.00512135773897171, -0.02847948856651783, -0.014144953340291977, 0.02871120162308216, -0.020295849069952965, 0.013460350222885609, 0.03722134232521057, -0.007988791912794113, -0.001346824923530221, 0.020464366301894188, 0.039243556559085846, 0.027468383312225342, -0.02173878252506256, 0.02348715253174305, 0.018663333728909492, -0.009647637605667114, -0.06112978979945183, 0.02407696470618248, 0.04191877320408821, -0.007714950479567051, 0.011827834881842136, -0.005792795680463314, -0.07499036937952042, -0.04490996524691582, 0.03810606151819229, 0.056200649589300156, 0.020801402628421783, -0.0056927381083369255, 0.009368530474603176, -0.023613540455698967, -0.0008590452489443123, -0.018410557880997658, 0.00031284388387575746, -0.013281300663948059, 0.017146674916148186, -0.00030494460952468216, 0.008599667809903622, -0.00736211659386754, -0.028795460239052773, -0.06664874404668808, -0.05101872980594635, 0.011185361072421074, 0.021317487582564354, 0.00282530440017581, -0.012112208642065525, -0.001302720746025443, -0.004713228903710842, 0.0209277905523777, -0.026225565001368523, 0.004678999073803425, -0.06336265057325363, 0.05388353019952774, 0.05316733196377754, 0.024919552728533745, -0.044151633977890015, 0.03905397281050682, 0.02628875896334648, ...]"
1,"August: Vincent Kompany became the first Belgian to manage in the Premier League, while he became just the third manager to face a team he previously played for in the competition in his maiden fixture (also Roberto Di Matteo for West Brom v Chelsea in August 2010 and Scott Parker for Fulham v Chelsea in March 2019), with all three losing those games.","[-0.021047059446573257, -0.0463116392493248, 0.0290360189974308, -0.024757668375968933, -0.041850797832012177, -0.0023546144366264343, 0.04533836618065834, -0.012429525144398212, -0.011679292656481266, 0.03157058730721474, 0.033476583659648895, -0.00022034907306078821, -0.023318031802773476, -0.021817568689584732, 0.01346362940967083, 0.024068264290690422, -0.06257343292236328, 0.05328677222132683, -0.04241854324936867, -0.017640598118305206, 0.01907009445130825, 0.04134388640522957, 0.030070124194025993, 0.04558168351650238, 0.012388971634209156, 0.0015803036512807012, -0.025690387934446335, -0.0420130118727684, -0.060505226254463196, 0.005454392172396183, 0.0503263957798481, -0.030637867748737335, -0.007051170337945223, 0.012946576811373234, 0.06743980199098587, -0.019688529893755913, -0.02398715913295746, -0.015389901585876942, 0.02504153922200203, 0.002141710603609681, 0.010422146879136562, -0.01625165529549122, 0.016028612852096558, -0.004029964096844196, -0.038221295922994614, 0.0010645188158378005, -0.0023596833925694227, 0.03256414085626602, -0.01654566451907158, 0

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [13]:
def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    # Get embeddings for the question text
    question_embeddings = get_embeddings(question, EMBEDDINGS_MODEL)[0]
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()  
    df_copy['distances'] = df_copy['embeddings'].apply(lambda embedding: cosine(embedding, question_embeddings))
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order
    df_copy.sort_values('distances', ascending=True, inplace=True)
    return df_copy

In [14]:
def create_context_prompt(question, df, max_token_count=1500):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """

    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    prompt = '''
    Use the context below consisting of some facts about the 2023-24 English Premier League season to answer the subsequent question. If the answer cannot be found, write "I don't know."

    Context:
    \"\"\"
    {}
    \"\"\"

    Question: {}'''

    # Count the number of tokens in the prompt template and question
    current_token_count = len(tokenizer.encode(prompt)) + len(tokenizer.encode(question))
    context = []
    for text in get_rows_sorted_by_relevance(question, df)['text'].values:
        text_token_count = len(tokenizer.encode(text))
        # Increase the counter based on the number of tokens in this row
        current_token_count += text_token_count
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt.format(''.join(context), question)

In [15]:
#Function to create a basic prompt containing the given question
def create_simple_prompt(question):
    prompt = '''

    Question: {}

    Answer: 
    '''
    return prompt.format(question)

In [16]:
def answer_question(prompt):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    response = client.chat.completions.create(
        messages=[
            {'role': 'system', 'content': 'You answer questions about the 2023-24 English Premier League season.'},
            {'role': 'user', 'content': prompt},
        ],
        model=GPT_MODEL,
    )

    return response.choices[0].message.content

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

In [67]:
question_1 = 'Has any player scored 10+ goals and provided 10+ assists in three consecutive english premier league seasons?'
print('Answer without context: ', answer_question(create_simple_prompt(question_1)))

Answer without context:  As of the 2023-24 Premier League season, no player has scored 10 or more goals and provided 10 or more assists in three consecutive seasons. While there have been players who have consistently performed at a high level, achieving this particular milestone over three successive seasons is quite rare in the league's history. Players like Kevin De Bruyne and Mohamed Salah have come close, but not specifically in three consecutive seasons with that exact statistic.


In [26]:
print('Answer with context: ', answer_question(create_context_prompt(question_1, df)))

Answer with context:  Yes, Liverpool’s Mohamed Salah became the first player in Premier League history to score 10+ goals and provide 10+ assists in three consecutive seasons.


In [32]:
question_2 = "Has any chelsea player scored a first half hatrick in the premier league?"
print('Answer without context: ', answer_question(create_simple_prompt(question_2)))

Answer without context:  As of October 2023, there has been no record of a Chelsea player scoring a first-half hat-trick specifically in the Premier League. However, players from Chelsea have achieved hat-tricks in Premier League matches, but details about specific instances of first-half hat-tricks may vary. For the most accurate and updated statistics, it is always best to check the official Premier League records or Chelsea FC's official communications.


In [33]:
print('Answer with context: ', answer_question(create_context_prompt(question_2, df)))

Answer with context:  Yes, Cole Palmer became the first Chelsea player to score a first half hat-trick in the Premier League during the 2023-24 season against Everton.


In [34]:
question_3 = "Which goalkeeper has scored the most own goals in the premier league?"
print('Answer without context: ', answer_question(create_simple_prompt(question_3)))

Answer without context:  As of the 2023-24 Premier League season, the record for the most own goals scored by a goalkeeper in the Premier League is held by Peter Schmeichel, who scored 4 own goals during his career in the league.


In [36]:
print('Answer with context: ', answer_question(create_context_prompt(question_3, df)))

Answer with context:  Emiliano Martínez has scored the most own goals in Premier League history, with three own goals.


In [45]:
question_4 = "Who has the most premier league appearances for Everton?"
print('Answer without context: ', answer_question(create_simple_prompt(question_4)))

Answer without context:  The player with the most Premier League appearances for Everton is Gareth Barry. He made a total of 422 appearances for the club in the Premier League during his time at Everton from 2013 to 2018.


In [44]:
print('Answer with context: ', answer_question(create_context_prompt(question_4, df)))

Answer with context:  Séamus Coleman has the most Premier League appearances for Everton, having made his 355th appearance for the club.


In [63]:
question_5 = "Who is the youngest player to score in four consecutive Premier League appearances for Manchester United?"
print('Answer without context: ', answer_question(create_simple_prompt(question_5)))

Answer without context:  The youngest player to score in four consecutive Premier League appearances for Manchester United is Mason Greenwood.


In [64]:
print('Answer with context: ', answer_question(create_context_prompt(question_5, df)))

Answer with context:  Rasmus Højlund is the youngest player to score in four consecutive Premier League appearances for Manchester United.
